A system of five linear equations has the following expression in matrix notation:
$$
\mathbf{y} = \mathbf{W}\mathbf{x}
$$

with
* $\mathbf{x}$ is a column vector representing our inputs whose size is (5,1).
* $\mathbf{y}$ is a column vector representing our outputs whose size is (5,1).
* $\mathbf{W}$ is a matrix representing our coefficients whose size is (5,5).

In [1]:
!pip install memtorch-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 24.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 12.0 MB/s eta 0:00:00
  Created wheel for memtorch-cpu: filename=memtorch_cpu-1.1.6-cp310-cp310-linux_x86_64.whl size=17099747 sha256=10194edecd90dc59bde7f1cd032b99f2924e68c85e515ca07d72b424910060d3
  Stored in directory: /root/.cache/pip/wheels/a0/44/56/544bd473d871cd13451bb8fa34ed88a025c1485c7dfdb236f5
  Created wheel for sklearn: filename=sklearn-0.0.post9-py3-none-any.whl size=2952 sha256=20c18cb491133f2fb64aa06dbcee0d2768c7488875b813464f739dd4780a48b7
  Stored in directory: /root/.cache/pip/wheels/33/a3/d2/092b519e9522b4c91608b7dcec0dd9051fa1bff4c45f4502d1
Successfully built memtorch-cpu sklearn


# Experiment

In [2]:
import torch
import torch.nn as nn
import numpy as np

class LinearEquations(nn.Module):
    def __init__(self):
        super(LinearEquations, self).__init__()

    def forward(self, A, B):
      self.A = A
      self.B = B
      return np.linalg.solve(self.A,self.B)

model = LinearEquations()

A = np.array([[8, 3, -2], [-4, 7, 5], [3, 4, -12]])
B = np.array([9, 15, 35])

output = model.forward(A, B)
print(output)

[-0.58226371  3.22870478 -1.98599767]


In [3]:
import memtorch
import copy
from memtorch.mn.Module import patch_model
from memtorch.map.Input import naive_scale
from memtorch.map.Parameter import naive_map

device = torch.device('cpu' if 'cpu' in memtorch.__version__ else 'cuda')
model = LinearEquations().to(device)

reference_memristor = memtorch.bh.memristor.VTEAM
reference_memristor_params = {'time_series_resolution': 1e-10}
memristor = reference_memristor(**reference_memristor_params)

patched_model = patch_model(copy.deepcopy(model),
                          memristor_model=reference_memristor,
                          memristor_model_params=reference_memristor_params,
                          module_parameters_to_patch=[torch.nn.Conv2d],
                          mapping_routine=naive_map,
                          transistor=True,
                          programming_routine=None,
                          tile_shape=(128, 128),
                          max_input_voltage=0.3,
                          scaling_routine=naive_scale,
                          ADC_resolution=8,
                          ADC_overflow_rate=0.,
                          quant_method = 'linear')
patched_model.tune_()